Most of these cropping algos were found from: https://stackoverflow.com/questions/13538748/crop-black-edges-with-opencv

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

imageToUse = samplefiles[5]

# img = cv2.imread(imageToUse)
# gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
# contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
# cnt = contours[0]
# x,y,w,h = cv2.boundingRect(cnt)
# crop = img[y:y+h,x:x+w]
# cv2.imwrite('cropped.png',crop)

def blackandWhite():
    padding = 0
    img = cv2.imread(imageToUse)
    bw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rows, cols = bw.shape

    non_empty_columns = np.where(bw.max(axis=0) > 0)[0]
    non_empty_rows = np.where(bw.max(axis=1) > 0)[0]
    cropBox = (min(non_empty_rows) * (1 - padding),
                min(max(non_empty_rows) * (1 + padding), rows),
                min(non_empty_columns) * (1 - padding),
                min(max(non_empty_columns) * (1 + padding), cols))

    return img[cropBox[0]:cropBox[1]+1, cropBox[2]:cropBox[3]+1 , :]
crop = blackandWhite()
cv2.imwrite('cropped.png',crop)

#----------------------------------------------
def trim_recursive(frame):
  if frame.shape[0] == 0:
    return np.zeros((0,0,3))

  # crop top
  if not np.sum(frame[0]):
    return trim_recursive(frame[1:])
  # crop bottom
  elif not np.sum(frame[-1]):
    return trim_recursive(frame[:-1])
  # crop left
  elif not np.sum(frame[:, 0]):
    return trim_recursive(frame[:, 1:])
    # crop right
  elif not np.sum(frame[:, -1]):
    return trim_recursive(frame[:, :-1])
  return frame
img = cv2.imread(imageToUse)
# thold = (img>120)*img
img = trim_recursive(img)
cv2.imwrite('cropped1.png',img)
#--------------------------------------------------
def trim_contours(frame):
  gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
  _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #_, 
  if len(contours) == 0:
    return np.zeros((0,0,3))
  cnt = contours[0]
  x, y, w, h = cv2.boundingRect(cnt)
  crop = frame[y:y + h, x:x + w]
  return crop
img = cv2.imread(imageToUse)
img = trim_contours(img)
cv2.imwrite('cropped2.png',img)
#-------------------------------------------------
def trim_contours_exact(frame):
  gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
  _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)#  _, 
  if len(contours) == 0:
    return np.zeros((0,0,3))
  cnt = contours[0]
  x, y, w, h = cv2.boundingRect(cnt)
  crop = frame[y:y + h, x:x + w]
  return crop

img = cv2.imread(imageToUse)
img = trim_contours_exact(img)
cv2.imwrite('cropped3.png',img)
#-------------------------------------------------

def trim_iterative(frame):
  for start_y in range(1, frame.shape[0]):
    if np.sum(frame[:start_y]) > 0:
      start_y -= 1
      break
  if start_y == frame.shape[0]:
    if len(frame.shape) == 2:
      return np.zeros((0,0))
    else:
      return np.zeros((0,0,0))
  for trim_bottom in range(1, frame.shape[0]):
    if np.sum(frame[-trim_bottom:]) > 0:
      break

  for start_x in range(1, frame.shape[1]):
    if np.sum(frame[:, :start_x]) > 0:
      start_x -= 1
      break
  for trim_right in range(1, frame.shape[1]):
    if np.sum(frame[:, -trim_right:]) > 0:
      break

  end_y = frame.shape[0] - trim_bottom + 1
  end_x = frame.shape[1] - trim_right + 1

  # print('iterative cropping x:{}, w:{}, y:{}, h:{}'.format(start_x, end_x - start_x, start_y, end_y - start_y))
  return frame[start_y:end_y, start_x:end_x]

img = cv2.imread(imageToUse)
img = trim_iterative(img)
cv2.imwrite('cropped4.png',img)
#-------------------------------------------------
def autocrop(image, threshold=0):
  """Crops any edges below or equal to threshold

  Crops blank image to 1x1.

  Returns cropped image.

  """
  if len(image.shape) == 3:
    flatImage = np.max(image, 2)
  else:
    flatImage = image
  assert len(flatImage.shape) == 2

  rows = np.where(np.max(flatImage, 0) > threshold)[0]
  if rows.size:
    cols = np.where(np.max(flatImage, 1) > threshold)[0]
    image = image[cols[0]: cols[-1] + 1, rows[0]: rows[-1] + 1]
  else:
    image = image[:1, :1]

  return image

img = cv2.imread(imageToUse)
img = autocrop(img)
cv2.imwrite('cropped5.png',img)
#-------------------------------------------------
def crop(image):#this one--does quite well
    y_nonzero, x_nonzero, _ = np.nonzero(image>20)
    return image[np.min(y_nonzero):np.max(y_nonzero), np.min(x_nonzero):np.max(x_nonzero)]
img = cv2.imread(imageToUse)
img = crop(img)
cv2.imwrite('cropped6.png',img)


img=mpimg.imread('./cropped6.png')
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.show()
#-------------------------------------------------
#This one was also experimented although didnt have any success
# https://github.com/linchundan88/Fundus-image-preprocessing


Found that the algorithm used in a dataset:
https://www.kaggle.com/datasets/tanlikesmath/diabetic-retinopathy-resized
and the specific code to crop was from this notebook:
https://www.kaggle.com/code/tanlikesmath/diabetic-retinopathy-resnet50-binary-cropped/notebook

worked the best in comparison to all of the tested cropping and resizing algorithms.

In [ ]:
# import the necessary packages
import numpy as np
import cv2
import glob
import os
from tqdm import tqdm
import math
from PIL import Image
files = glob.glob('D:\\Experiments with Deep Learning\\DR Kaggle\\train\\train\\train\\*.jpeg')#point to the folder where the images are

new_sz = 1024

def crop_image(image):
    output = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,gray = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)
    contours,hierarchy = cv2.findContours(gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print('no contours!')
        flag = 0
        return image, flag
    cnt = max(contours, key=cv2.contourArea)
    ((x, y), r) = cv2.minEnclosingCircle(cnt)
    x = int(x); y = int(y); r = int(r)
    flag = 1
    #print(x,y,r)
    if r > 100:
        return output[0 + (y-r)*int(r<y):-1 + (y+r+1)*int(r<y),0 + (x-r)*int(r<x):-1 + (x+r+1)*int(r<x)], flag
    else:
        print('none!')
        flag = 0
        return image,flag

#resize - set
for i in tqdm(range(len(files))):
    img = cv2.imread(files[i])
    img_cropped, flag = crop_image(img)
    if not flag:
        print(files[i])
    height, width, _= img_cropped.shape
    ratio = height/width
    if width > new_sz:
        new_image = cv2.resize(img_cropped,(new_sz,math.ceil(new_sz*ratio)))  
    else:
        new_image = img_cropped
    cv2.imwrite('D:\\Experiments with Deep Learning\\DR Kaggle\\train\\train\\resized_train_cropped\\'+os.path.basename(files[i]),new_image)#point to the folder where you want to save the resized images

Code to find the ratio to test:

In [ ]:
#do crop on all train to make new dataset - make csv
# %%time
#AutoCropping Train images. - takes 5 hr
import cv2
import pandas as pd
import os

trainImages =  pd.DataFrame(columns=['image', 'ratio_after'])

#autocropping
def crop(image):#define crop function to test
    y_nonzero, x_nonzero, _ = np.nonzero(image>20)#threshold here!!!!!
    return image[np.min(y_nonzero):np.max(y_nonzero), np.min(x_nonzero):np.max(x_nonzero)]

#create df and save csv to report with ratio before and after
for i, image in enumerate(os.listdir('../input/resized-2015-2019-blindness-detection-images/resized train 15/')):#for each train image
#   print(image)
  try:
   
    img = cv2.imread('../input/resized-2015-2019-blindness-detection-images/resized train 15/'+ image)#read it from filepath
    h, w, _ = img.shape
    ratio = str(w/h)
    new_row = {'image':image ,'ratio_after': ratio}
    trainImages = trainImages.append(new_row, ignore_index=True)
 
    print(f"Converted: {i+1} images, latest img name: {image}, it had ratio after of: {w/h}, width and height {w}, {h}")
  except ValueError:  #raised if `img` is empty.
    print(f"Error with image: {image}, array zero?")
    pass

#save csv
trainImages.to_csv('image_cropping.csv', index = False)#save Dataframe to csv